In [1]:

######## snakemake preamble start (automatically inserted, do not edit) ########
library(methods)
Snakemake <- setClass(
    "Snakemake",
    slots = c(
        input = "list",
        output = "list",
        params = "list",
        wildcards = "list",
        threads = "numeric",
        log = "list",
        resources = "list",
        config = "list",
        rule = "character",
        bench_iteration = "numeric",
        scriptdir = "character",
        source = "function"
    )
)
snakemake <- Snakemake(
    input = list('data/Methylation_predictions_train_20_predict_80/Methylation_pred_lasso_normalized_trimmed_formatted.rds', "input_file" = 'data/Methylation_predictions_train_20_predict_80/Methylation_pred_lasso_normalized_trimmed_formatted.rds'),
    output = list('data/Methylation_predictions_train_20_predict_80/Full_data_Methylation_pred_lasso_formatted_standardized.rds', "output_formatted_pred" = 'data/Methylation_predictions_train_20_predict_80/Full_data_Methylation_pred_lasso_formatted_standardized.rds'),
    params = list(),
    wildcards = list(),
    threads = 1,
    log = list('logs/processed_notebooks/Methylation_standardizing_predictions.r.ipynb', "notebook" = 'logs/processed_notebooks/Methylation_standardizing_predictions.r.ipynb'),
    resources = list('mem_mb', 'disk_mb', 'tmpdir', 'time', "mem_mb" = 200000, "disk_mb" = 1000, "tmpdir" = '/scratch/57589791', "time" = '20:00:00'),
    config = list(),
    rule = 'methylation_standardize',
    bench_iteration = as.numeric(NA),
    scriptdir = '/faststorage/project/DELFI1/Workspaces/CarmenAndAnika/prep_formatting_normalizing',
    source = function(...){
        wd <- getwd()
        setwd(snakemake@scriptdir)
        source(...)
        setwd(wd)
    }
)
setwd('/faststorage/project/DELFI1/Workspaces/CarmenAndAnika');

######## snakemake preamble end #########


## Standardizing the ATAC predictions 

__This notebook takes an input file name (ATAC predictions made for all individudals) and an output file name. The input file is read in, predictions are standardized row by row and the standardized predictions are written to the output file.__

In [2]:
# packages

library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.2
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [3]:
#input_filename <- "../data/ATAC_predictions_on_all_samples_NEW/ATAC_pred_lasso_std_TRUE_gaussian_formatted.csv"
#output_filename <- "../data/ATAC_predictions_on_all_samples_NEW/ATAC_pred_lasso_std_TRUE_gaussian_formatted_standardized.csv"

input_filename <- snakemake@input[["input_file"]]
output_filename <-  snakemake@output[["output_formatted_pred"]]

In [4]:
#df <- read.csv(input_filename)
df <- readRDS(input_filename)
head(df)
dim(df)

ERROR while rich displaying an object: Error in sprintf(wrap, header, body): 'fmt' length exceeds maximal format length 8192

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. repr::mime2repr[[mime]](obj)
8. 

sample,chr10_400,chr10_40,chr10_4,chr10_401,chr10_402,chr10_403,chr10_404,chr10_405,chr10_406,...,chr9_39,chr9_391,chr9_392,chr9_393,chr9_394,chr9_395,chr9_396,chr9_397,chr9_398,chr9_399
PGDX16568P,50.10720,38.43050,50.49563,36.26990,37.08899,35.73950,42.94098,50.30837,50.66313,...,41.69085,48.46803,39.19122,40.33245,49.63915,42.58743,52.08225,51.84441,53.84734,35.06284
PGDX16569P,50.38967,43.69226,53.56724,40.19999,40.67496,45.66306,45.94454,53.63420,54.25514,...,39.99292,49.49178,46.14314,43.29620,50.04835,46.53519,51.45761,49.33736,63.15294,43.94147
PGDX16570P,54.09841,38.90010,57.60327,39.01121,42.97006,43.96420,46.19221,59.91990,57.36566,...,42.65936,49.98062,50.99560,44.71672,47.49022,51.49670,55.23129,52.10197,64.84029,47.99915
PGDX16571P,54.90257,40.47710,55.52637,41.71517,42.23254,43.34020,47.61174,58.27360,56.45344,...,48.34666,51.90264,55.36530,48.01901,52.85030,49.18811,59.65190,59.16895,64.22539,51.64244
PGDX16579P,54.40836,35.16049,57.19377,36.97275,38.29034,39.15255,47.17557,51.75797,52.33919,...,45.87922,49.45779,47.96223,40.21917,45.66219,48.05531,60.02450,58.92792,56.02913,43.20545
PGDX16580P,54.04343,40.79401,54.85352,36.91397,39.41826,41.45725,45.73760,56.09565,53.96090,...,43.38544,49.79330,50.40758,43.99041,51.16138,51.78069,57.08762,54.30238,64.64102,44.41425


[1]   424 10020

In [5]:
df_samples <- df %>% select(sample)
head(df_samples)

df <- df %>% select(-sample)
df_transposed <- as.data.frame(t(df))
head(df_transposed)
dim(df_transposed)


sample
PGDX16568P
PGDX16569P
PGDX16570P
PGDX16571P
PGDX16579P
PGDX16580P


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V415,V416,V417,V418,V419,V420,V421,V422,V423,V424
chr10_400,50.10720,50.38967,54.09841,54.90257,54.40836,54.04343,59.01541,48.96306,42.22359,50.19156,...,54.82127,50.47315,59.61169,61.03471,59.15501,110.58755,87.25414,87.88410,58.77866,97.29948
chr10_40,38.43050,43.69226,38.90010,40.47710,35.16049,40.79401,43.47191,39.36098,32.16328,39.72207,...,42.38241,42.72863,55.99356,46.59339,43.81425,94.17728,85.84339,77.57071,48.56021,92.54186
chr10_4,50.49563,53.56724,57.60327,55.52637,57.19377,54.85352,60.44841,59.01224,53.75023,58.24836,...,58.75053,55.25458,63.49777,62.75198,66.71542,113.33267,95.68346,97.06109,64.03625,110.20699
chr10_401,36.26990,40.19999,39.01121,41.71517,36.97275,36.91397,45.82643,39.33831,43.58036,33.98321,...,41.02241,45.29979,58.54923,52.02448,49.47813,88.96391,90.41875,72.40543,50.21012,95.85642
chr10_402,37.08899,40.67496,42.97006,42.23254,38.29034,39.41826,49.29339,39.20427,36.85384,37.58302,...,45.25225,39.92613,55.81549,51.78548,46.51996,94.08180,80.74828,76.80742,50.97758,90.99163
chr10_403,35.73950,45.66306,43.96420,43.34020,39.15255,41.45725,47.18420,40.29256,40.06383,45.39868,...,40.96200,36.90759,54.74666,49.57037,51.00325,78.41047,84.24743,75.54391,52.81437,95.55913


[1] 10019   424

In [6]:
df_transposed_scaled <- scale(df_transposed)
head(df_transposed_scaled)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V415,V416,V417,V418,V419,V420,V421,V422,V423,V424
chr10_400,0.6438705,0.3680161,0.3925523,0.5205899,0.5811027,0.5863300,0.4954924,0.008541025,-0.6510246,0.09096211,...,0.5072341,0.3396912,-0.2521960,0.5052355,0.3838524,0.4491621,-0.34689706,0.2737527,0.03350801,-0.6531363
chr10_40,-0.6658968,-0.4473249,-1.2688220,-0.9713554,-1.2765581,-0.8459487,-1.0195553,-1.022028456,-1.7962904,-0.91929111,...,-0.7623493,-0.6668116,-0.8384101,-1.1374052,-1.1522902,-0.8741337,-0.48922783,-0.9214095,-1.03481880,-1.2118072
chr10_4,0.6874403,0.7548527,0.7756788,0.5851056,0.8499291,0.6739017,0.6351688,1.087096487,0.6611679,0.86840193,...,0.9082778,0.9611011,0.3774307,0.7005680,1.1409126,0.6705240,0.50353986,1.3372233,0.58318281,0.8625481
chr10_401,-0.9082499,-0.8724721,-1.2566765,-0.8433086,-1.1016520,-1.2653854,-0.7900568,-1.024462303,-0.4965707,-1.47306135,...,-0.9011584,-0.3326548,-0.4243372,-0.5196405,-0.5851397,-1.2945314,-0.02761914,-1.5199860,-0.86232255,-0.8225889
chr10_402,-0.8163731,-0.8146499,-0.8239221,-0.7898002,-0.9744882,-0.9946688,-0.4521271,-1.038848651,-1.2623175,-1.12569847,...,-0.4694365,-1.0310327,-0.8672605,-0.5468262,-0.8813551,-0.8818329,-1.00327499,-1.0098629,-0.78208581,-1.3938444
chr10_403,-0.9677447,-0.2073996,-0.7152500,-0.6752408,-0.8912743,-0.7742520,-0.6577135,-0.922044255,-0.8968921,-0.37152706,...,-0.9073248,-1.4233321,-1.0404338,-0.7987859,-0.4324221,-2.1455423,-0.65024527,-1.1562846,-0.59005106,-0.8574996


In [7]:
apply(df_transposed_scaled, 2, mean)

V1            V2            V3            V4            V5 
-7.630961e-17  3.230134e-16  3.030025e-16  3.142826e-16  1.568676e-16 
           V6            V7            V8            V9           V10 
 2.126936e-16 -1.418380e-16 -1.871232e-16  5.026231e-17 -1.399533e-16 
          V11           V12           V13           V14           V15 
 1.616492e-16 -1.994620e-16 -8.554003e-17  1.976342e-16  1.137400e-16 
          V16           V17           V18           V19           V20 
-1.754026e-16  2.722474e-16  2.348308e-16  1.226312e-16  4.040037e-16 
          V21           V22           V23           V24           V25 
-6.368037e-17  1.276627e-16 -2.057072e-16 -3.101679e-17 -3.164648e-16 
          V26           V27           V28           V29           V30 
 3.526249e-16 -2.043488e-16  3.300177e-16  2.871195e-16  3.344416e-16 
          V31           V32           V33           V34           V35 
-4.152853e-16  4.472305e-16 -9.073140e-17 -1.695118e-16  2.612033e-16 
          V36           V37           V38           V39           V40 
 3.082559e-16 -1.408668e-16  2.512670e-16  1.000771e-16 -8.542785e-17 
          V41           V42           V43           V44           V45 
-3.773155e-16 -1.859319e-16  2.988576e-17  1.729742e-16 -1.083667e-16 
          V46           V47           V48           V49           V50 
 1.838461e-17  4.710733e-17  3.255099e-16 -1.134268e-16 -1.173693e-16 
          V51           V52           V53           V54           V55 
-1.234595e-16 -3.375590e-16 -3.135864e-16 -9.689657e-17 -3.250741e-17 
          V56           V57           V58           V59           V60 
-1.437110e-16  2.045026e-16 -3.205917e-17 -2.363584e-16  6.345229e-17 
          V61           V62           V63           V64           V65 
 3.955626e-16 -1.115335e-16  3.022102e-16 -3.895647e-16  1.350526e-16 
          V66           V67           V68           V69           V70 
 2.605510e-16  9.817401e-17  1.222711e-18  3.716955e-16 -1.897059e-16 
          V71           V72           V73           V74           V75 
-6.341397e-17  3.177728e-16  2.616491e-16  4.709523e-16  2.315023e-16 
          V76           V77           V78           V79           V80 
-2.844202e-16 -1.433582e-16  3.136310e-16  3.926301e-17  2.043248e-17 
          V81           V82           V83           V84           V85 
-3.133162e-16  3.657456e-16 -6.307237e-17 -1.908351e-16  1.614638e-16 
          V86           V87           V88           V89           V90 
 4.909411e-17 -6.505867e-17 -1.170640e-16 -6.056588e-17 -3.985061e-17 
          V91           V92           V93           V94           V95 
-2.237959e-16 -7.006704e-17 -2.484931e-16 -1.914277e-16  5.943713e-17 
          V96           V97           V98           V99          V100 
-4.563024e-17  1.955609e-17  6.750341e-18 -1.445443e-16 -3.192682e-16 
         V101          V102          V103          V104          V105 
 1.772045e-16 -3.512978e-16 -2.674105e-16  1.664553e-16  1.460649e-16 
         V106          V107          V108          V109          V110 
 2.864268e-16 -1.414220e-16  2.093300e-16  3.819632e-16  1.470442e-17 
         V111          V112          V113          V114          V115 
 1.112592e-16  2.030890e-16  5.078680e-17  1.948655e-16  9.066001e-17 
         V116          V117          V118          V119          V120 
 2.745729e-16 -3.104324e-16 -2.061726e-16  3.472024e-16 -1.333535e-16 
         V121          V122          V123          V124          V125 
-2.263753e-16  4.055704e-17 -3.550033e-16 -2.763881e-16 -3.518871e-16 
         V126          V127          V128          V129          V130 
 3.409307e-16 -2.298390e-16  5.505425e-17  2.685506e-16 -4.183903e-16 
         V131          V132          V133          V134          V135 
-1.194140e-16 -4.369429e-17  3.229449e-16  1.211331e-16  1.278121e-16 
         V136          V137          V138          V139          V140 
-1.273644e-16 -1.233555e-16  1.000959e-16  1.111050e-16  1.718088e-16 
         V141          V142        

In [8]:
apply(df_transposed_scaled, 2, sd)

V1   V2   V3   V4   V5   V6   V7   V8   V9  V10  V11  V12  V13  V14  V15  V16 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
 V17  V18  V19  V20  V21  V22  V23  V24  V25  V26  V27  V28  V29  V30  V31  V32 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
 V33  V34  V35  V36  V37  V38  V39  V40  V41  V42  V43  V44  V45  V46  V47  V48 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
 V49  V50  V51  V52  V53  V54  V55  V56  V57  V58  V59  V60  V61  V62  V63  V64 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
 V65  V66  V67  V68  V69  V70  V71  V72  V73  V74  V75  V76  V77  V78  V79  V80 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
 V81  V82  V83  V84  V85  V86  V87  V88  V89  V90  V91  V92  V93  V94  V95  V96 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
 V97  V98  V99 V100 V101 V102 V103 V104 V105 V106 V107 V108 V109 V110 V111 V112 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V113 V114 V115 V116 V117 V118 V119 V120 V121 V122 V123 V124 V125 V126 V127 V128 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V129 V130 V131 V132 V133 V134 V135 V136 V137 V138 V139 V140 V141 V142 V143 V144 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V145 V146 V147 V148 V149 V150 V151 V152 V153 V154 V155 V156 V157 V158 V159 V160 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V161 V162 V163 V164 V165 V166 V167 V168 V169 V170 V171 V172 V173 V174 V175 V176 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V177 V178 V179 V180 V181 V182 V183 V184 V185 V186 V187 V188 V189 V190 V191 V192 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V193 V194 V195 V196 V197 V198 V199 V200 V201 V202 V203 V204 V205 V206 V207 V208 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V209 V210 V211 V212 V213 V214 V215 V216 V217 V218 V219 V220 V221 V222 V223 V224 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V225 V226 V227 V228 V229 V230 V231 V232 V233 V234 V235 V236 V237 V238 V239 V240 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V241 V242 V243 V244 V245 V246 V247 V248 V249 V250 V251 V252 V253 V254 V255 V256 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V257 V258 V259 V260 V261 V262 V263 V264 V265 V266 V267 V268 V269 V270 V271 V272 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V273 V274 V275 V276 V277 V278 V279 V280 V281 V282 V283 V284 V285 V286 V287 V288 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V289 V290 V291 V292 V293 V294 V295 V296 V297 V298 V299 V300 V301 V302 V303 V304 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V305 V306 V307 V308 V309 V310 V311 V312 V313 V314 V315 V316 V317 V318 V319 V320 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V321 V322 V323 V324 V325 V326 V327 V328 V329 V330 V331 V332 V333 V334 V335 V336 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V337 V338 V339 V340 V341 V342 V343 V344 V345 V346 V347 V348 V349 V350 V351 V352 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V353 V354 V355 V356 V357 V358 V359 V360 V361 V362 V363 V364 V365 V366 V367 V368 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V369 V370 V371 V372 V373 V374 V375 V376 V377 V378 V379 V380 V381 V382 V383 V384 
   1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 
V385 V386 V387 V388 V389 V390 V391 V392 V393 V394 V395 V396 V397 V398 V399 V400 
   1    1    1    1    1    1    

In [9]:
df_scaled <-  as.data.frame(t(df_transposed_scaled))
head(df_scaled)
dim(df_scaled)

ERROR while rich displaying an object: Error in sprintf(wrap, header, body): 'fmt' length exceeds maximal format length 8192

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. repr::mime2repr[[mime]](obj)
8. 

,chr10_400,chr10_40,chr10_4,chr10_401,chr10_402,chr10_403,chr10_404,chr10_405,chr10_406,chr10_407,...,chr9_39,chr9_391,chr9_392,chr9_393,chr9_394,chr9_395,chr9_396,chr9_397,chr9_398,chr9_399
V1,0.6438705,-0.6658968,0.6874403,-0.9082499,-0.8163731,-0.9677447,-0.1599592,0.6664361,0.7062285,0.5087841,...,-0.3001858,0.46000554,-0.58056717,-0.4525564,0.5913699,-0.19961676,0.8654100,0.8387317,1.0633995,-1.0436448
V2,0.3680161,-0.4473249,0.7548527,-0.8724721,-0.8146499,-0.2073996,-0.1731334,0.7630041,0.8385967,0.3311739,...,-0.8976809,0.25870700,-0.14895590,-0.4955413,0.3264639,-0.10122724,0.4980273,0.2399086,1.9218133,-0.4169858
V3,0.3925523,-1.2688220,0.7756788,-1.2566765,-0.8239221,-0.7152500,-0.4716988,1.0289169,0.7497046,0.8192595,...,-0.8578863,-0.05757607,0.05337401,-0.6329901,-0.3298093,0.10815039,0.5163901,0.1743151,1.5667805,-0.2741766
V4,0.5205899,-0.9713554,0.5851056,-0.8433086,-0.7898002,-0.6752408,-0.2334600,0.8692367,0.6809880,0.7634747,...,-0.1574507,0.21032423,0.56844786,-0.1913386,0.3083347,-0.07042508,1.0117861,0.9618375,1.4847965,0.1834134
V5,0.5811027,-1.2765581,0.8499291,-1.1016520,-0.9744882,-0.8912743,-0.1169525,0.3253061,0.3814019,1.2825685,...,-0.2420664,0.10331075,-0.04103000,-0.7883324,-0.2630127,-0.03204632,1.1231298,1.0172960,0.7375265,-0.5001185
V6,0.5863300,-0.8459487,0.6739017,-1.2653854,-0.9946688,-0.7742520,-0.3115400,0.8081783,0.5774085,0.4844657,...,-0.5658114,0.12688551,0.19329076,-0.5004138,0.2747775,0.34172520,0.9154107,0.6143236,1.7319427,-0.4545966


[1]   424 10019

In [10]:
df_scaled <- cbind(df_samples, df_scaled)
head(df_scaled)
dim(df_scaled)

ERROR while rich displaying an object: Error in sprintf(wrap, header, body): 'fmt' length exceeds maximal format length 8192

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. repr::mime2repr[[mime]](obj)
8. 

,sample,chr10_400,chr10_40,chr10_4,chr10_401,chr10_402,chr10_403,chr10_404,chr10_405,chr10_406,...,chr9_39,chr9_391,chr9_392,chr9_393,chr9_394,chr9_395,chr9_396,chr9_397,chr9_398,chr9_399
V1,PGDX16568P,0.6438705,-0.6658968,0.6874403,-0.9082499,-0.8163731,-0.9677447,-0.1599592,0.6664361,0.7062285,...,-0.3001858,0.46000554,-0.58056717,-0.4525564,0.5913699,-0.19961676,0.8654100,0.8387317,1.0633995,-1.0436448
V2,PGDX16569P,0.3680161,-0.4473249,0.7548527,-0.8724721,-0.8146499,-0.2073996,-0.1731334,0.7630041,0.8385967,...,-0.8976809,0.25870700,-0.14895590,-0.4955413,0.3264639,-0.10122724,0.4980273,0.2399086,1.9218133,-0.4169858
V3,PGDX16570P,0.3925523,-1.2688220,0.7756788,-1.2566765,-0.8239221,-0.7152500,-0.4716988,1.0289169,0.7497046,...,-0.8578863,-0.05757607,0.05337401,-0.6329901,-0.3298093,0.10815039,0.5163901,0.1743151,1.5667805,-0.2741766
V4,PGDX16571P,0.5205899,-0.9713554,0.5851056,-0.8433086,-0.7898002,-0.6752408,-0.2334600,0.8692367,0.6809880,...,-0.1574507,0.21032423,0.56844786,-0.1913386,0.3083347,-0.07042508,1.0117861,0.9618375,1.4847965,0.1834134
V5,PGDX16579P,0.5811027,-1.2765581,0.8499291,-1.1016520,-0.9744882,-0.8912743,-0.1169525,0.3253061,0.3814019,...,-0.2420664,0.10331075,-0.04103000,-0.7883324,-0.2630127,-0.03204632,1.1231298,1.0172960,0.7375265,-0.5001185
V6,PGDX16580P,0.5863300,-0.8459487,0.6739017,-1.2653854,-0.9946688,-0.7742520,-0.3115400,0.8081783,0.5774085,...,-0.5658114,0.12688551,0.19329076,-0.5004138,0.2747775,0.34172520,0.9154107,0.6143236,1.7319427,-0.4545966


[1]   424 10020

In [11]:
#write.csv(df_scaled, output_filename,  row.names = FALSE)
saveRDS(df_scaled, file = output_filename)